# Can we identify faulty or suboptimally performing equipment?

In [1]:
#setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

gd=pd.read_csv("Data/Test_Plant1.csv")

In [2]:
#Add columns
gd['DC_POWER_DIF_AVG_PER_ALL']=gd['DC_POWER']-gd['DC_POWER'].mean()
gd['AC_POWER_DIF_AVG_PER_ALL']=gd['AC_POWER']-gd['AC_POWER'].mean()
gd['DATE_TIME'] = pd.to_datetime(gd['DATE_TIME'], errors='coerce')
gd['DATE']=pd.to_datetime(gd['DATE_TIME'].dt.date)
gd['TIME']=gd['DATE_TIME'].dt.time

In [3]:
#New columns per inverter 
uurimiseks= gd.copy()
groups = dict(list(uurimiseks.groupby("SOURCE_KEY")))
shift_nr=3
shift_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']
gr_nr=0

#inverter based shifting
for group in groups:
    gr_nr+=1
    dates=groups[group]['DATE'].unique()
    for i in range(shift_nr):
        for j in range(len(shift_columns)):
            groups[group][shift_columns[j]+'_PER_INVERTER_MOMENT_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=-(i+1), fill_value=0)
            groups[group][shift_columns[j]+'_PER_INVERTER_DAY_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=-(i+1)*96, fill_value=0)
    
    #groups[group]['INVERTER']=group - ebaoluline
    groups[group]['DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    ############ per one day comparison, but note sure if correct
    gr_no2=0
    for date in dates:
        gr_no2 +=1
        d=groups[group][groups[group]['DATE']==date]
        d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
        d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])
        if gr_no2==1:
            date_data=pd.DataFrame(columns = d.keys())
            
        date_data =pd.concat([date_data, d], ignore_index=True)
    #########
    
    if gr_nr==1:
        gen_data=pd.DataFrame(columns = date_data.keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data=pd.concat([gen_data,date_data], ignore_index=True)
    
#gen_data=gen_data.fillna(value=0) - ebaoluline
gen_data

<ipython-input-3-d3be87962ed3>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
<ipython-input-3-d3be87962ed3>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])


,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY
0,2020-06-09 12:00:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000
1,2020-06-09 12:15:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000
2,2020-06-09 12:30:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000
3,2020-06-09 12:45:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000
4,2020-06-09 13:00:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17947,2020-06-17 22:45:00,zVJPv84UY57bAof,22.150570,21.480377,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,21.969495,0.0,20.734963,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276
17948,2020-06-17 23:00:00,zVJPv84UY57bAof,22.129816,21.389024,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,21.909288,0.0,20.427972,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276
17949,2020-06-17 23:15:00,zVJPv84UY57bAof,22.008275,20.709211,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276
17950,2020-06-17 23:30:00,zVJPv84UY57bAof,21.969495,20.734963,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276


# Validation by comparison

In [4]:
gen_data['AC_POWER'][20000:20004]

Series([], Name: AC_POWER, dtype: float64)

In [5]:
gen_data['AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1'][20000:20004]

Series([], Name: AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1, dtype: float64)

In [6]:
a=gen_data[gen_data['SOURCE_KEY']=='1BY6WEcLGh8j5v7']
b=pd.concat([a[a['DATE_TIME']=='2020-05-17 13:00:00'],a[a['DATE_TIME']=='2020-05-16 13:00:00']], ignore_index=True)
b['AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1']

Series([], Name: AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1, dtype: float64)

In [7]:
b['AC_POWER']

Series([], Name: AC_POWER, dtype: float64)

In [8]:
groups[group].keys()

Index(['DATE_TIME', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE',
       'IRRADIATION', 'PLANT_ID', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD',
       'TOTAL_YIELD', 'ARTIFICIAL', 'DC_POWER_DIF_AVG_PER_ALL',
       'AC_POWER_DIF_AVG_PER_ALL', 'DATE', 'TIME',
       'DC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
     

# Other fields

In [9]:
#Group by day labels
'''
* label "FAULTY_according_to_DAILY_YIELD" - yield stays the same, while other inverters still increases
* label "INCORRECT_daily_yield" - yield per day decreases
* columns per inverter day average differences (total are done)
'''

uurimiseks= gen_data.copy()
groups = dict(list(uurimiseks.groupby("DATE")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_AVG_DIF_PER_DAY','AC_AVG_DIF_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_AVG_DIF_PER_DAY'] =  groups[group]['DC_POWER'] -np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_AVG_DIF_PER_DAY'] =  groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    gr_nr+=1
    '''
    Tundub et toodaks vale infot
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_DAY_SHIFT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data2=pd.DataFrame(columns =groups[group].keys())
    
    gen_data2=pd.concat([gen_data2,groups[group]], ignore_index=True)
    
gen_data2

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY
0,2020-06-09 12:00:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000,0.000000,0.00000
1,2020-06-09 12:15:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000,0.000000,0.00000
2,2020-06-09 12:30:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000,0.000000,0.00000
3,2020-06-09 12:45:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000,0.000000,0.00000
4,2020-06-09 13:00:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17947,2020-06-17 22:45:00,zVJPv84UY57bAof,22.150570,21.480377,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,20.734963,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276,-2517.888173,-246.39372
17948,2020-06-17 23:00:00,zVJPv84UY57bAof,22.129816,21.389024,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,20.427972,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276,-2517.888173,-246.39372
17949,2020-06-17 23:15:00,zVJPv84UY57bAof,22.008275,20.709211,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,0.000000,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276,-2517.888173,-246.39372
17950,2020-06-17 23:30:00,zVJPv84UY57bAof,21.969495,20.734963,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,0.000000,0.0,0.0,0.0,-1736.338441,-169.866850,-2530.472545,-247.621276,-2517.888173,-246.39372


In [10]:
groups[group].keys()

Index(['DATE_TIME', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE',
       'IRRADIATION', 'PLANT_ID', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD',
       'TOTAL_YIELD', 'ARTIFICIAL', 'DC_POWER_DIF_AVG_PER_ALL',
       'AC_POWER_DIF_AVG_PER_ALL', 'DATE', 'TIME',
       'DC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'DAILY_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'TOTAL_YIELD_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
       'MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_1',
       'IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_1',
     

In [11]:
# add inverter difference from average per that moment
'''
* columns "MOMENT_AVERAGE" - average value per this moment of inverter production info
* columns "DIF_FROM_AVERAGE" - inverter avg values in comparison with other inverters per this moment
'''

uurimiseks= gen_data2.copy()
groups = dict(list(uurimiseks.groupby("DATE_TIME")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_DIF_AVG_PER_DAY','AC_DIF_AVG_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_DIF_AVG_PER_MOMENT'] =  groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_MOMENT'] =  groups[group]['AC_POWER'] -np.mean(groups[group]['AC_POWER']) 

    gr_nr+=1
    ''' 
    tundub valet tulemust andvat
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_MOMENT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data3=pd.DataFrame(columns =groups[group].keys())
    
    gen_data3=pd.concat([gen_data3,groups[group]], ignore_index=True)
    
gen_data3

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY,DC_DIF_AVG_PER_MOMENT,AC_DIF_AVG_PER_MOMENT
0,2020-06-09 12:00:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000,0.000000,0.00000,0.0,0.0
1,2020-06-09 12:00:00,1IF53ai7Xc0U56Y,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,-1784.059550,-174.517810,0.000000,0.000000,0.000000,0.00000,0.0,0.0
2,2020-06-09 12:00:00,3PZuoBAID5Wc2HD,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,-1761.561222,-172.308554,0.000000,0.000000,0.000000,0.00000,0.0,0.0
3,2020-06-09 12:00:00,7JYdWkrLSPkdwr4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,-1736.582475,-169.888539,0.000000,0.000000,0.000000,0.00000,0.0,0.0
4,2020-06-09 12:00:00,McdE0feGgRqW7Ca,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0,-1777.303129,-173.852736,0.000000,0.000000,0.000000,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17947,2020-06-17 23:45:00,uHbuxQJl8lW7ozc,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5967.000,7287002.0,...,0.0,0.0,-1748.068582,-170.989951,-2556.190960,-250.116879,-2517.888173,-246.39372,0.0,0.0
17948,2020-06-17 23:45:00,wCURE6d3bPkepu2,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5147.625,7028601.0,...,0.0,0.0,-1754.324628,-171.614827,-2516.066071,-246.225078,-2517.888173,-246.39372,0.0,0.0
17949,2020-06-17 23:45:00,z9Y9gH1T5YWrNuG,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5819.000,7251204.0,...,0.0,0.0,-1729.627679,-169.211636,-2477.912277,-242.506961,-2517.888173,-246.39372,0.0,0.0
17950,2020-06-17 23:45:00,zBIq5rxdHJRwDNY,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5817.000,6583369.0,...,0.0,0.0,-1718.205383,-168.082481,-2488.603646,-243.529178,-2517.888173,-246.39372,0.0,0.0


In [12]:
#other labels adding
'''
* label "INCORRECT_total_yield" - yield per period decreases
* label "INCORRECT_according_to_DC_vs_AC" - DC and AC ratio is off
* label "INCORRECT_according_to_DC_AC_DAILY_YIELD" - DC and AC increases, while DAILY_YIELD does not
'''

'\n* label "INCORRECT_total_yield" - yield per period decreases\n* label "INCORRECT_according_to_DC_vs_AC" - DC and AC ratio is off\n* label "INCORRECT_according_to_DC_AC_DAILY_YIELD" - DC and AC increases, while DAILY_YIELD does not\n'

In [13]:
gen_data3.to_csv(path_or_buf='Data/Test_Plant1_added_features.csv')

In [14]:
katse=pd.read_csv("Data/Test_Plant1_added_features.csv")

In [15]:
katse

,Unnamed: 0,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,...,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY,DC_DIF_AVG_PER_MOMENT,AC_DIF_AVG_PER_MOMENT
0,0,2020-06-09 12:00:00,1BY6WEcLGh8j5v7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,-1480.230580,-144.868409,0.000000,0.000000,0.000000,0.00000,0.0,0.0
1,1,2020-06-09 12:00:00,1IF53ai7Xc0U56Y,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,-1784.059550,-174.517810,0.000000,0.000000,0.000000,0.00000,0.0,0.0
2,2,2020-06-09 12:00:00,3PZuoBAID5Wc2HD,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,-1761.561222,-172.308554,0.000000,0.000000,0.000000,0.00000,0.0,0.0
3,3,2020-06-09 12:00:00,7JYdWkrLSPkdwr4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,-1736.582475,-169.888539,0.000000,0.000000,0.000000,0.00000,0.0,0.0
4,4,2020-06-09 12:00:00,McdE0feGgRqW7Ca,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,...,0.0,0.0,-1777.303129,-173.852736,0.000000,0.000000,0.000000,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17947,17947,2020-06-17 23:45:00,uHbuxQJl8lW7ozc,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5967.000,...,0.0,0.0,-1748.068582,-170.989951,-2556.190960,-250.116879,-2517.888173,-246.39372,0.0,0.0
17948,17948,2020-06-17 23:45:00,wCURE6d3bPkepu2,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5147.625,...,0.0,0.0,-1754.324628,-171.614827,-2516.066071,-246.225078,-2517.888173,-246.39372,0.0,0.0
17949,17949,2020-06-17 23:45:00,z9Y9gH1T5YWrNuG,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5819.000,...,0.0,0.0,-1729.627679,-169.211636,-2477.912277,-242.506961,-2517.888173,-246.39372,0.0,0.0
17950,17950,2020-06-17 23:45:00,zBIq5rxdHJRwDNY,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5817.000,...,0.0,0.0,-1718.205383,-168.082481,-2488.603646,-243.529178,-2517.888173,-246.39372,0.0,0.0
